Part 1: Image alignment

In [1]:
import cv2
import numpy as np

# Load image
m1 = cv2.imread('D://m1.tif')
m2 = cv2.imread('D://m2.tif')
m3 = cv2.imread('D://m3.tif')

# Create SIFT detector
sift = cv2.SIFT_create()

def align_images(im1, im2):
    # Convert to grayscale image
    gray1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Detect feature points and descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

    # Create a BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # Match descriptors
    matches = bf.match(descriptors1, descriptors2)

    # Sort by distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract coordinates of matched points
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # calculate a homography matrix
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Transform the image using the homologous matrix
    height, width, channels = im2.shape
    im1_aligned = cv2.warpPerspective(im1, h, (width, height))

    return im1_aligned

# Align images
m1_aligned = align_images(m1, m2)
m2_aligned = m2 # m2 is the reference image
m3_aligned = align_images(m3, m2)

# Save aligned images
cv2.imwrite('D://m1_aligned.tif', m1_aligned)
cv2.imwrite('D://m2_aligned.tif', m2_aligned)
cv2.imwrite('D://m3_aligned.tif', m3_aligned)

# Separate images to red, yellow and green channels, and save each channel
for i, aligned_image in enumerate([m1_aligned, m2_aligned, m3_aligned], start=1):
    zeros = np.zeros_like(aligned_image[:, :, 0])
    for j, channel in enumerate(['b', 'g', 'r']):
        color_image = np.stack([
            aligned_image[:, :, j] if k == j else zeros for k in range(3)
        ], axis=-1)
        cv2.imwrite(f'D://mm{i}-{channel}.tif', color_image)

Part 2: Target reconstruction

In [7]:
# Separate the channels for each image
def split_channels(image):
    r, g, b = cv2.split(image)
    return g, b, r  

# Extract the gray value for each channel. y, yellow, Cy3; g, green, Fam; r, red, Cy5
Image1_y = cv2.imread('D://mm1-b.tif', cv2.IMREAD_GRAYSCALE)
Image1_g = cv2.imread('D://mm1-g.tif', cv2.IMREAD_GRAYSCALE)
Image1_r = cv2.imread('D://mm1-r.tif', cv2.IMREAD_GRAYSCALE)

Image2_y = cv2.imread('D://mm2-b.tif', cv2.IMREAD_GRAYSCALE)
Image2_g = cv2.imread('D://mm2-g.tif', cv2.IMREAD_GRAYSCALE)
Image2_r = cv2.imread('D://mm2-r.tif', cv2.IMREAD_GRAYSCALE)

Image3_y = cv2.imread('D://mm3-b.tif', cv2.IMREAD_GRAYSCALE)
Image3_g = cv2.imread('D://mm3-g.tif', cv2.IMREAD_GRAYSCALE)
Image3_r = cv2.imread('D://mm3-r.tif', cv2.IMREAD_GRAYSCALE)


# Get the size of each image
height, width = Image1_g.shape

# Create an empty matrix to store the new gray value
new_gray_values = np.zeros((height, width), dtype=np.uint8)

# Traverse each pixel coordinate
for i in range(height):
    for j in range(width):
        # Calculate the new gray value according to the pre-set color barcode (the following is just an example for SV2)
        if Image1_g[i, j] > Image2_g[i, j]:
            new_gray_values[i, j] = Image1_g[i, j] - Image2_g[i, j]
        else:
            new_gray_values[i, j] = 0
# Convert the new grayscale value matrix to an image
cv2.imwrite('D://SV2.tif', new_gray_values)

True